In [8]:
import requests
import json
import os
import pandas
import util

In [9]:
TOKEN = ""
BEACON_QUERY_URL = "https://beacon-cmems.maris.nl/api/query"

In [ ]:
query_parameters = []

# TIME
query_parameters.append(util.column("TIME", "COMMON_TIME"))
query_parameters.append(util.column("TIME_QC", "COMMON_TIME_QC"))

# LONGITUDE
query_parameters.append(util.column("LONGITUDE"))
query_parameters.append(util.column("LONGITUDE.units"))

# COMMON LONGITUDE
query_parameters.append(util.column("LONGITUDE", "COMMON_LONGITUDE"))
query_parameters.append(util.value("degrees_east", "COMMON_LONGITUDE_UNITS"))
query_parameters.append(util.value("LONGITUDE", "COMMON_LONGITUDE_STANDARD_NAME"))
query_parameters.append(util.value("SDN:P01::ALONZZ01", "COMMON_LONGITUDE_P01"))
query_parameters.append(util.value("SDN:P06::DEGE", "COMMON_LONGITUDE_P06"))

# LATITUDE
query_parameters.append(util.column("LATITUDE"))
query_parameters.append(util.column("LATITUDE.units"))
# COMMON LATITUDE
query_parameters.append(util.column("LATITUDE", "COMMON_LATITUDE"))
query_parameters.append(util.value("degrees_north", "COMMON_LATITUDE_UNITS"))
query_parameters.append(util.value("LATITUDE", "COMMON_LATITUDE_STANDARD_NAME"))
query_parameters.append(util.value("SDN:P01::ALATZZ01", "COMMON_LATITUDE_P01"))
query_parameters.append(util.value("SDN:P06::DEGN", "COMMON_LATITUDE_P06"))

# POSITION QC
query_parameters.append(util.column("POSITION_QC"))
query_parameters.append(util.column("POSITION_QC", "COMMON_POSITION_QC"))

# COMMON DEPTH (NEEDS TO BE ADJUSTED)
# query_parameters.append(util.column(util.coalesce_columns(["PRES_ADJUSTED","DEPH_ADJUSTED","PRES","DEPH"]), "COMMON_ORIGIN_DEPTH"))
# query_parameters.append(util.column(util.coalesce_columns(["PRES_ADJUSTED_QC","DEPH_ADJUSTED_QC","PRES_QC","DEPH_QC"]), "COMMON_ORIGIN_DEPTH_QC"))
# query_parameters.append(util.column(util.coalesce_columns(["PRES_ADJUSTED.units","DEPH_ADJUSTED.units","PRES.units","DEPH.units"]), "COMMON_ORIGIN_DEPTH_UNITS"))
# query_parameters.append(util.value("Depth", "COMMON_ORIGIN_DEPTH_STANDARD_NAME"))
# query_parameters.append(util.value("SDN:P01::ADEPZZ01", "COMMON_ORIGIN_DEPTH_P01"))
# query_parameters.append(util.value("SDN:P06::ULAA", "COMMON_ORIGIN_DEPTH_P06"))

query_parameters.append(
    util.coalesce_columns(
        [
            util.function_call(
                "pressure_to_depth",
                [util.coalesce_columns(["PRES_ADJUSTED", "PRES"]), "LATITUDE"],
            ),
            "DEPH",
        ],
        "COMMON_DEPTH",
    )
)
query_parameters.append(util.coalesce_columns("m", "COMMON_DEPTH_UNITS"))
query_parameters.append(
    util.coalesce_columns(
        ["PRES_ADJUSTED_QC", "ADJUSTED_DEPH_QC", "PRES_QC", "DEPH_QC"],
        "COMMON_DEPTH_QC",
    )
)
query_parameters.append(util.value("m", "COMMON_DEPTH_UNITS"))
query_parameters.append(util.value("Depth", "COMMON_DEPTH_STANDARD_NAME"))
query_parameters.append(util.value("SDN:P01::ADEPZZ01", "COMMON_DEPTH_P01"))
query_parameters.append(util.value("SDN:P06::ULAA", "COMMON_DEPTH_P06"))

In [ ]:
# MAP CHLOROPHYLL
query_parameters.append(util.column("CPHL"))
query_parameters.append(util.column("CPHL_QC"))
query_parameters.append(util.column("CPHL.units"))
query_parameters.append(util.column("CPHL.standard_name"))

query_parameters.append(util.column("CHLT"))
query_parameters.append(util.column("CHLT_QC"))
query_parameters.append(util.column("CHLT.units"))
query_parameters.append(util.column("CHLT.standard_name"))

# MAP COMMON CHLOROPHYLL
query_parameters.append(
    util.column(
        util.coalesce_columns(["CPHL", "CHLT"]), "COMMON_CHLOROPHYLL_PER_VOLUME"
    )
)
query_parameters.append(
    util.column(
        util.coalesce_columns(["CPHL_QC", "CHLT_QC"]),
        "COMMON_CHLOROPHYLL_PER_VOLUME_QC",
    )
)
query_parameters.append(
    util.value(
        "Concentration of chlorophyll per unit volume of the water column",
        "COMMON_CHLOROPHYLL_PER_VOLUME_STANDARD_NAME",
    )
)
query_parameters.append(util.value("mg/m^3", "COMMON_CHLOROPHYLL_PER_VOLUME_UNITS"))
query_parameters.append(
    util.value("SDN:P01::CHLTVOLU", "COMMON_CHLOROPHYLL_PER_VOLUME_P01")
)
query_parameters.append(
    util.value("SDN:P06::UMMC", "COMMON_CHLOROPHYLL_PER_VOLUME_P06")
)

In [ ]:
# MAP OXYGEN
query_parameters.append(util.column("DOXY"))
query_parameters.append(util.column("DOXY_QC"))
query_parameters.append(util.column("DOXY.units"))
query_parameters.append(util.column("DOXY.standard_name"))

query_parameters.append(util.column("DOX1"))
query_parameters.append(util.column("DOX1_QC"))
query_parameters.append(util.column("DOX1.units"))
query_parameters.append(util.column("DOX1.standard_name"))

query_parameters.append(util.column("DOX2"))
query_parameters.append(util.column("DOX2_QC"))
query_parameters.append(util.column("DOX2.units"))
query_parameters.append(util.column("DOX2.standard_name"))

# MAP COMMON OXYGEN VOLUME
query_parameters.append(
    util.column(
        util.coalesce_columns(["DOXY", "DOX1", "DOX2"]), "COMMON_OXYGEN_PER_VOLUME"
    )
)
query_parameters.append(
    util.column(
        util.coalesce_columns(["DOXY_QC", "DOX1_QC", "DOX2_QC"]),
        "COMMON_OXYGEN_PER_VOLUME_QC",
    )
)
query_parameters.append(
    util.value(
        "Concentration of chlorophyll per unit volume of the water column",
        "COMMON_OXYGEN_PER_VOLUME_STANDARD_NAME",
    )
)
query_parameters.append(util.value("mg/m^3", "COMMON_OXYGEN_PER_VOLUME_UNITS"))
query_parameters.append(util.value("SDN:P01::CHLTVOLU", "COMMON_OXYGEN_PER_VOLUME_P01"))
query_parameters.append(util.value("SDN:P06::UMMC", "COMMON_OXYGEN_PER_VOLUME_P06"))